# Pricing Options using Bayesian Neural Networks

**Names:** Andrew Zabelo, Daniel Wen, Kyle McCandless  
**Student IDs:** 2176083, 2159859, 2157818

## Setup

### Imports and Helper Functions

In [5]:
# General
import os
import random
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import statsmodels.api as sm

# ML tools
import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split

# BNN specific
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
from pyro.infer import MCMC, NUTS
from pyro.infer import Predictive

# We'll suppress some warnings from tensorflow, but feel free to
# comment out this line and see some of the efficiency gains we can get!
warnings.filterwarnings("ignore")

In [6]:
# Note: All io_fns and model_fns start with an 'io_' and 'model_' prefix
# respectively
def fitting_returns_data(data_path,
                         io_fn,
                         model_fn,
                         seed = None,
                         print_summary = True):
    """
    Fits data to supplied model and provides returns and alpha estimates.

        Parameters:
            data_path (str): path to factor data with first column as date
                6 factors, and the last column as risk free rate
            io_fn (fn): a function to calculate input/output pairs for the
                model fit
            model_fn (fn): a function that takes in input data, output data,
                and training and testing values, and returns the strategy
                for each day, and the observed return for each day
            seed (int): if given, the seed for the model fit will be set to
                this value
            print_summary (bool): if True, then the model won't print a summary
                of the OLS fit for our strategy return.
        Returns:
            strat_df (pd.DataFrame): The weights we would give each factor
                on each day
            return_vector (list): The returns from our strategy on each day
            model_OLS (RegressionResultsWrapper): A regression calculating the
                alpha for our strategy
    """
    # if given, set the random seed
    if seed:
        set_seed(seed)

    # Load the data into a pandas DataFrame
    data = pd.read_csv(data_path)
    # Drop date and risk free rate
    data = data.iloc[:, 1:7]

    # Shift the data by one time step to create input/output pairs
    X, y = io_fn(data)

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

    # Fit the model
    strat_df, return_vector = model_fn(X, y, X_train, y_train, X_val, y_val)

    # Calculate alpha
    y_ols = sm.add_constant(y)
    model_OLS = sm.OLS(return_vector, y_ols).fit()
    if print_summary:
        print(model_OLS.summary())

    return strat_df, return_vector, model_OLS

def set_seed(seed_value):
    # Adding a fixed seed from this solution: https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras
    # 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed_value)

    # 2. Set the `python` built-in pseudo-random generator at a fixed value
    random.seed(seed_value)

    # 3. Set the `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)

    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    tf.compat.v1.set_random_seed(seed_value)
    return

def io_day_1_lag(data):
    # Create input output pairs, where the input is the previous day of data
    # and the output is the current day of data.
    X = data.shift(1).dropna().reset_index(drop=True)
    y = data.dropna().iloc[1:,:].reset_index(drop=True)
    return X, y

def io_day_5_lag(data):
    X = data.shift(1).add_suffix('_lag1')
    for i in range(2, 6):
        shifted_data = data.shift(i).add_suffix('_lag{}'.format(i))
        X = pd.concat([X, shifted_data], axis=1)
    X = X.dropna().reset_index(drop=True)
    y = data.iloc[5:,:].reset_index(drop=True)
    return X, y

def io_day_1_lag_second_order_input(data):
    # Create input output pairs where input data includes second order interactions
    X, y = io_day_1_lag(data)
    cols = X.columns

    for i in range(len(cols)):
        for j in range(i, len(cols)):
            col_name = cols[i] + cols[j]
            col_values = X[cols[i]] * X[cols[j]]
            X[col_name] = col_values
    return X, y

def model_feed_forward(X, y, X_train, y_train, X_val, y_val):
    # Define the neural network model
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(6, activation='linear'))

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Train the model, verbose = 0 means reports aren't printed
    # at the end of each epoch
    model.fit(X_train, y_train, batch_size=32, epochs=50,
              validation_data=(X_val, y_val))

    # Make predictions
    predictions = model.predict(X)

    pred_df = pd.DataFrame(predictions)

    return predictions_to_returns(pred_df, y)

def predictions_to_returns(pred_df, y):
    # Given the predictions of each factor for each day, calculate our
    # strategy for each day, and the returns for each day

    # Apply our strategy to our predictions
    strat_df = pred_df.apply(lambda row : max_predicted_factor_strat(row), axis = 1)

    # Calculate our returns
    return_vector = np.multiply(strat_df,np.asarray(y)).apply(sum, axis = 1)

    return strat_df, return_vector

def predictions_to_returns_max_sharpe(pred_df, std_df, y):
    # Given the predictions of each factor for each day, calculate our
    # strategy for each day, and the returns for each day

    # Apply our strategy to our predictions
    strat_df = pred_df.apply(lambda row : max_predicted_factor_strat(row), axis = 1)

    # Calculate our returns
    return_vector = np.multiply(strat_df,np.asarray(y)).apply(sum, axis = 1)

    return strat_df, return_vector

def max_predicted_factor_strat(row):
    # For each day, set our strategy to be the factor with
    # the highest predicted return
    max_pred_return = max(row)
    row_list = [x == max_pred_return for x in row]
    return pd.Series(row_list)

def max_sharpe_ratio_factor_strat(row):
    # For each day, set our strategy to be the factor with
    # the highest predicted sharpe ratio
    max_pred_return = max(row)
    row_list = [x == max_pred_return for x in row]
    return pd.Series(row_list)

## Predicting Factor returns using a Bayesian Neural Network

### Define the BNN

In [7]:
class MyFirstBNN(PyroModule):
    def __init__(self, in_dim=6, out_dim=6, hid_dim=10, prior_scale=1.):
        super().__init__()

        self.activation = nn.Tanh()  # or nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](in_dim, hid_dim)  # Input to hidden layer
        self.layer2 = PyroModule[nn.Linear](hid_dim, out_dim)  # Hidden to output layer

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim, in_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim]).to_event(1))
        self.layer2.weight = PyroSample(dist.Normal(0., prior_scale).expand([out_dim, hid_dim]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim]).to_event(1))

    def forward(self, x, y=None):
        x = self.activation(self.layer1(x))
        mu = self.layer2(x).squeeze()
        sigma = pyro.sample("sigma", dist.Gamma(0.5, 2.0))  # Infer the response noise

        # Sampling model
        with pyro.plate("data", 6):
            obs = pyro.sample("obs", dist.Normal(mu, sigma*sigma), obs=y)
        return mu
    
# Calculate returns and alphas using BNN with 5 day lag
def model_bnn(X, y, X_train, y_train, X_val, y_val):
    # Convert data
    x_train_tensor = torch.tensor(X_train.values).float()
    y_train_tensor = torch.tensor(y_train.values).float()
    x_val_tensor = torch.tensor(X_val.values).float()
    x_val_tensor = torch.tensor(y_val.values).float()
    x_tensor = torch.tensor(X.values).float()
    y_tensor = torch.tensor(y.values).float()

    # Define the neural network model
    model = MyFirstBNN()

    # Set Pyro random seed
    pyro.set_rng_seed(COMMON_SEED)

    # Define Hamiltonian Monte Carlo (HMC) kernel
    # NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
    nuts_kernel = NUTS(model, jit_compile=True) # jit_compile=True is faster but requires PyTorch 1.6+

    # Define MCMC sampler, get 50 posterior samples
    mcmc = MCMC(nuts_kernel, num_samples=15)

    # Run MCMC
    mcmc.run(x_train_tensor, y_train_tensor)

    # Make predictions
    predictive = Predictive(model=model, posterior_samples=mcmc.get_samples())
    preds = predictive(x_tensor)
    
    pred_means = preds['obs'].T.detach().numpy().mean(axis=2)
    pred_df = pd.DataFrame(pred_means.T)
    
    pred_stds = preds['obs'].T.detach().numpy().std(axis=2)
    std_df = pd.DataFrame(pred_stds.T)

    return predictions_to_returns(pred_df, y)
#     return predictions_to_returns_max_sharpe(pred_df, std_df, y)
    

### Train the BNN and calculate alphas

In [8]:
COMMON_SEED = 1

fitting_returns_data(
    'ff6_factors_19630701_20230131.csv',
    io_day_1_lag,
    model_bnn,
    seed = COMMON_SEED)

Sample: 100%|████| 30/30 [03:56,  7.87s/it, step size=9.78e-04, acc. prob=0.977]


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     421.6
Date:                Thu, 06 Jun 2024   Prob (F-statistic):               0.00
Time:                        00:46:40   Log-Likelihood:                -15192.
No. Observations:               14998   AIC:                         3.040e+04
Df Residuals:                   14991   BIC:                         3.045e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0456      0.005      8.349      0.0

(           0      1      2      3      4      5
 0      False  False  False  False  False   True
 1      False  False  False  False  False   True
 2      False   True  False  False  False  False
 3      False   True  False  False  False  False
 4      False  False   True  False  False  False
 ...      ...    ...    ...    ...    ...    ...
 14993  False  False  False  False  False   True
 14994  False   True  False  False  False  False
 14995  False  False  False   True  False  False
 14996  False  False   True  False  False  False
 14997  False  False   True  False  False  False
 
 [14998 rows x 6 columns],
 0        0.42
 1        0.41
 2        0.09
 3        0.07
 4        0.09
          ... 
 14993    0.14
 14994   -0.62
 14995   -0.61
 14996    0.72
 14997   -0.06
 Length: 14998, dtype: float64,
 <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x29e9580d0>)